In [20]:
# parte gráfica
import PySimpleGUI as sg
import pandas as pd     # importing pandas to write SharePoint list in excel or csv
from shareplum import Site 
from requests_ntlm import HttpNtlmAuth
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime
import plotly
import re
from datetime import date
import os

lista_datas = []
hj = date.today()
for x in range(1,12):
    lista_datas.append(str(date.fromordinal(hj.toordinal()+x)))
    
print=sg.Print
sg.theme('DarkBlue12')

menu_def = [      
                ['Ajuda', 'Instruções'], ]    
layout = [
        [sg.Menu(menu_def, tearoff=False)],
        [sg.Text('Preencha as informações abaixo:',size=(25, 1))],
        [sg.Text('Login', size=(5, 1), auto_size_text=False, justification='left'),sg.Input(size=(18, 1),key= 'login')],
        [sg.Text('Senha', size=(5, 1), justification='left'),sg.InputText(size=(18, 1),key= 'senha')],
        [sg.Text('Data da Prévia', size=(11, 1)),sg.Drop(values=(lista_datas),key='Data_previa', auto_size_text=True)],
        [sg.Text('_'  * 40)], 
        [sg.Button('Gerar prévia')]
]


# Display the window and get values

window = sg.Window('Prévia de pagamentos', layout)
event, values = window.read()
datafiltro_previa = values['Data_previa']
login = values['login']
senha = values['senha']

while True:
    event, value = window.read()
    if event == 'Gerar prévia':
        inicio = ('  Horário de início {}:{} - {}/{}/{} '.format(datetime.now().hour, datetime.now().minute,datetime.now().day,datetime.now().month,datetime.now().year ))
        print(inicio)
        print('[+] Importando dados do Sharepoint, por favor aguarde...')

        cred = HttpNtlmAuth(login, senha)
        site = Site('http://sharepoint/local', auth=cred)
        site2 = Site('http://sharepoint/local', auth=cred)
        site3 = Site('http://sharepoint/local', auth=cred)
        site4 = Site('http://sharepoint/local', auth=cred)
        caminho = 'C:\\local'
        caminho_dados_SAP = 'local\Dados SAP.xlsm'
        caminho_dados_fato = 'local\Previa.xlsx'


        lista_tributos = site.List('Guia Digital') # this creates SharePlum object
        lista_demais_areas = site.List('DEMAIS ÁREAS')
        lista_patrimonio = site2.List('Patrimonio')
        lista_pppagamento = site3.List('Liberacao PP')
        lista_excecoes = site4.List('Prévia')

        data_tributos = lista_tributos.GetListItems() # this will retrieve all items from list
        data_demais_areas = lista_demais_areas.GetListItems()
        data_patrimonio = lista_patrimonio.GetListItems()
        data_pppagamento = lista_pppagamento.GetListItems()
        data_excecoes = lista_excecoes.GetListItems()

        Tributos = pd.DataFrame(data_tributos[0:])
        Demais_areas = pd.DataFrame(data_demais_areas[0:])
        Patrimonio = pd.DataFrame(data_patrimonio[0:])
        Pp_pagamento = pd.DataFrame(data_pppagamento[0:])
        Excecoes = pd.DataFrame(data_excecoes[0:])
        print('   [-] Dados do Sharepoint importados com sucesso... ')
        print('[+] Importando dados do SAP, por favor aguarde...')
        SAP = pd.read_excel(caminho_dados_SAP)
        Fato = pd.read_excel(caminho_dados_fato)
        print('   [-] Dados do SAP importados com sucesso... ')
        
        Tributos = Tributos[['DATA DE PGTO','VALOR','EMPRESA']]
        Tributos.insert(loc=2, column='Area', value='Tributos') 
        Patrimonio =  Patrimonio[['DATA DE PGTO','VALOR','EMPRESA']]
        Patrimonio.insert(loc=2, column='Area', value='Patrimônio') 
        Demais_areas = Demais_areas[['DATA DE PGTO','VALOR','ÁREA','EMPRESA']]
        Pp_pagamento = Pp_pagamento[['Data de Pagamento','Valor Total','Empresa']]
        Pp_pagamento.insert(loc=2, column='Area', value='Pp_pagamento') 
        Excecoes = Excecoes[['Data de pagamento','Valor Total','Empresa']]
        Excecoes.insert(loc=2, column='Area', value='Exceções') 

        SAP = SAP[['data','V#proposta','Identificação','Empresa']]
        
        
        Tributos.set_axis(['Data_pgto', 'Valor', 'Area','Empresa'], axis=1, inplace=True)
        Demais_areas.set_axis(['Data_pgto', 'Valor', 'Area','Empresa'], axis=1, inplace=True)
        Patrimonio.set_axis(['Data_pgto', 'Valor', 'Area','Empresa'], axis=1, inplace=True)
        Pp_pagamento.set_axis(['Data_pgto', 'Valor', 'Area','Empresa'], axis=1, inplace=True)
        Excecoes.set_axis(['Data_pgto', 'Valor', 'Area','Empresa'], axis=1, inplace=True)
        SAP.set_axis(['Data_pgto', 'Valor', 'Area','Empresa'], axis=1, inplace=True)

        lista_dados = [Tributos,Demais_areas,Patrimonio,Pp_pagamento,Excecoes,SAP]
        df = pd.concat(lista_dados)
        df_full = df

        ano = str(datetime.now().year)+'-'
        mes = str(datetime.now().strftime("%m"))
        dia_hj = int(datetime.now().strftime("%d"))
        dia_amanha = str(dia_hj + 1)
        ano_passado = str((datetime.now().year) -1)
        datafiltro_previa_ano_passado = str(ano_passado+'-'+mes+'-'+dia_amanha)

        df.Valor = df.Valor.round(2)

        mask = (df['Data_pgto'] == datafiltro_previa)
        df=df.loc[mask].reset_index()
        def format(x):
            return "R$ {:,.2f}".format(x)


        mask = (df_full['Data_pgto'] >= datafiltro_previa_ano_passado)
        df_full=df_full.loc[mask].reset_index()
        mask = (df_full['Data_pgto'] <= datafiltro_previa)
        df_full=df_full.loc[mask].reset_index()

        df['Area'] = df['Area'].str.capitalize()
        df['Empresa'] = df['Empresa'].str.upper()
        df = pd.merge(Fato, df, on=['Area'], how='right')
        df.replace(to_replace ="Rh/brpe", 
                         value ="Rh",inplace=True)
        df.drop(columns=['index'])
        agrupado_empresa = df.groupby(['Empresa']).Valor.sum().round(2).reset_index()
        agrupado_categoria = df.groupby(['Categoria']).Valor.sum().round(2).reset_index()
        agrupado_categoria.sort_values(by='Valor',ascending=False,inplace = True)
        agrupado_empresa.sort_values(by='Valor',ascending=False,inplace = True)


        Area_table = list(agrupado_categoria.Categoria)
        Area_table.append('<b>Total</b>')
        Valor_table = list(agrupado_categoria.Valor.round(2))
        Valor_table.append(agrupado_categoria.Valor.sum().round(2))
        Valor_table = Valor_table = pd.DataFrame(Valor_table,columns=['Valor'])
        Valor_table = list(Valor_table.Valor.apply(format))
        pgto_table = list(agrupado_empresa.Empresa)
        pgto_table.append('<b>Total</b>')
        pgto_Valor_table = list(agrupado_empresa.Valor.round(2))
        pgto_Valor_table.append(agrupado_empresa.Valor.sum().round(2))
        pgto_Valor_table = pd.DataFrame(pgto_Valor_table,columns=['Valor'])
        pgto_Valor_table = list(pgto_Valor_table.Valor.apply(format))
        #--------------------------------------------------------------
        agrupado_empresa = df.groupby(['Empresa']).Valor.sum().round(2).reset_index()
        agrupado_categoria = df.groupby(['Categoria']).Valor.sum().round(2).reset_index()
        agrupado_categoria.sort_values(by='Valor',ascending=False,inplace = True)


        # carcap --------------------------------------
        print('[+] Importando dados do CarCap, por favor aguarde...')
        
        caminho_carcap = 'local\arrecad\\'
        diretorio = os.listdir(caminho_carcap)
        caminho_carcap = caminho_carcap + diretorio[0]
        carcap = pd.read_excel(caminho_carcap, sheet_name='RESUMO')
        carcap = carcap[3:12].reset_index()
        carcap.drop(columns=['index','Unnamed: 0','Unnamed: 2','Unnamed: 27'],inplace=True)
        carcap = carcap.dropna(subset=['Unnamed: 1'])
        print('   [-] Dados do Carcap importados com sucesso... ')
        carcap_1 = carcap
        carcap_1 = carcap_1.replace(np.nan, 0)
        nome_coluna = []
        ano = str(datetime.now().year)+'-'
        mes = str(datetime.now().strftime("%m"))

        for x in carcap_1.iloc[0]:
            x = str(x).replace(ano,'').replace(mes,'').replace(' 00:00:00','').replace('-','')
            if x == '':
                x = mes
            nome_coluna.append(x + '-' + mes)

        carcap_1.set_axis(nome_coluna, axis=1, inplace=True)
        carcap_1 = carcap_1[1:].reset_index()
        carcap_1.drop(columns=['index'],inplace=True)

        lista_colunas = list(carcap_1.columns)
        del lista_colunas[0]

        for defint in lista_colunas:
            carcap_1[defint] = carcap_1[defint].astype(int)

        carcap_2 = carcap_1.transpose()
        carcap_2.set_axis(carcap_2.iloc[0], axis=1, inplace=True)
        carcap_2 = carcap_2[1:]
        col1 = nome_coluna[0]
        del nome_coluna[0]
        carcap_2.insert(loc=0, column=col1, value=nome_coluna)
        totais_carcap = carcap_2[-3:]
        carcap_2 = carcap_2[carcap_2['(+) Arrecadação (Prévia)']!= 0]
        #filtra e ajusta data frame para nao aparecer datas indesejadas 
        lista_dt = datafiltro_previa.split(sep='-')
        data_fil = lista_dt[-1] + '-' + lista_dt[1] 
        mask2 = (carcap_2[carcap_2.columns[0]] <= data_fil) # filtra dia da previa e dias anteriores
        carcap_2= carcap_2.loc[mask2].reset_index()
        carcap_2.drop(columns=['index'],inplace = True)
        carcap_2.sort_values(by=carcap_2.columns[0],ascending=False,inplace = True)
        
        carcap_2 = pd.concat([carcap_2, totais_carcap])
        
        agrupado_categoria.sort_values(by='Valor',ascending=False,inplace = True)
        # fim carcap ------------------------------
        # dash parte vizual
        print('[+] Cruzamento de dados realizado...')
        print('[+] Gerando prévia...')


        agrupado_categoria.sort_values(by='Valor',ascending=False,inplace = True)

        agrupado_dia_um_ano = df_full.groupby(['Data_pgto']).Valor.sum().reset_index()

        fig = make_subplots(
            rows=4, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.03,
            specs=[[{"type": "table"}],
                   [{"type": "table"}],
                    [{"type": "table"}],
                   [{"type": "scatter"}]]
        )

        fig.add_trace(
            go.Table(
                header=dict(
                    values=list(agrupado_empresa.columns),
                    fill_color=[['orange']],
                    font=dict(size=12),
                    align="center"
                ),
                cells=dict(
                    values=[pgto_table,pgto_Valor_table],
                    fill_color='lavender',
                    align = "center")
            ),
            row=2, col=1

        )

        fig.add_trace(
            go.Table(
                header=dict(
                    values=list(agrupado_categoria.columns),
                    fill_color=[['orange']],
                    font=dict(size=12),
                    align="center"
                ),
                cells=dict(
                    values=[Area_table,Valor_table],
                    fill_color='lavender',
                    align = "center")
            ),
            row=3, col=1

        )

        fig.add_trace(
            go.Table(
                header=dict(
                    values=carcap_2.columns,
                    font=dict(size=10),
                    fill_color=['orange'],
                    align="center"
                ),
                cells=dict(
                    values=[carcap_2[k].tolist() for k in carcap_2.columns],
                    align = "center")
            ),
            row=1, col=1
        )

        fig.add_trace(
            go.Scatter(
                x=agrupado_dia_um_ano.Data_pgto,
                y=agrupado_dia_um_ano.Valor,
                mode="lines",
                marker_color='darkorange',
                name="hash-rate-TH/s"
            ),
            row=4, col=1
        )
        
        fig.update_layout(
            height=800,
            showlegend=False,
            title_text="Prévia de pagamentos " + lista_dt[-1] + '/' + lista_dt[1] + '/' + lista_dt[0],
        )

        fig.add_layout_image(
            dict(
                source="local/image",

                xref="paper", yref="paper",
                x=1.09, y=1,
                sizex=2, sizey=0.16,
                xanchor="right", yanchor="bottom"
            )
        )
        
        plotly.offline.plot(fig,filename='local\ report_day ' + lista_dt[-1] + '_' + lista_dt[1] + '_' + lista_dt[0] + '.html')
        sg.popup_notify('Prévia de pagamento gerada!')
        
        
        
        
    elif event == 'Instruções': 
        print('Para utilizar o programa de gerar prévia, siga as seguintes instruções :\n • Insira o caminho de rede aonde está salvo a base SAP. \n • Insira os arquivo aonde indica "Planilha Shaepoint".  \n • O arquivo será gerado em html')

        
    else:
        break
        window.close()

